In [27]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [28]:
combined_df = pd.read_csv('combined_dataset.csv')

In [29]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(combined_df['URL'])
X = tokenizer.texts_to_sequences(combined_df['URL'])
X = pad_sequences(X)

In [30]:
X

array([[    0,     0,     0, ...,  1491,    12,  1236],
       [    0,     0,     0, ...,    15,    12,   649],
       [    0,     0,     0, ...,  2205,  2205,    31],
       ...,
       [    0,     0,     0, ...,   286,    12,   144],
       [    0,     0,     0, ..., 18356, 30623, 78520],
       [    0,     0,     0, ..., 22896,    48,  5603]], dtype=int32)

In [31]:
y = (combined_df['Type'] == 'Malicious').astype(int)

In [32]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=27)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=27)

In [33]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=X.shape[1]))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [34]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [36]:
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping])


Epoch 1/100
774/774 [==============================] - 636s 816ms/step - loss: 0.0553 - accuracy: 0.9815 - val_loss: 0.1214 - val_accuracy: 0.9703
Epoch 2/100
774/774 [==============================] - 813s 1s/step - loss: 0.0063 - accuracy: 0.9991 - val_loss: 0.0033 - val_accuracy: 0.9991
Epoch 3/100
774/774 [==============================] - 695s 899ms/step - loss: 8.8104e-05 - accuracy: 1.0000 - val_loss: 0.0028 - val_accuracy: 0.9992
Epoch 4/100
774/774 [==============================] - 796s 1s/step - loss: 3.0593e-05 - accuracy: 1.0000 - val_loss: 0.0024 - val_accuracy: 0.9994
Epoch 5/100
774/774 [==============================] - 655s 846ms/step - loss: 1.5814e-05 - accuracy: 1.0000 - val_loss: 0.0023 - val_accuracy: 0.9995
Epoch 6/100
774/774 [==============================] - 641s 828ms/step - loss: 9.2258e-06 - accuracy: 1.0000 - val_loss: 0.0022 - val_accuracy: 0.9995
Epoch 7/100
774/774 [==============================] - 616s 796ms/step - loss: 5.6779e-06 - accuracy: 1.0000

In [37]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

332/332 [==============================] - 71s 213ms/step - loss: 4.1198e-04 - accuracy: 0.9997
Test loss: 0.00041197522659786046
Test accuracy: 0.9997173547744751


In [38]:
#model.save('raw_url_model.h5') 